In [12]:
#Importação das bibliotecas

import pandas as pd
from time import sleep
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions
import pyodbc

conn = pyodbc.connect('Driver={MySQL ODBC 8.0 ANSI Driver};'
                      'Server=tup02mql007;'
                      'Database=plan_mailing;'
                      'UID=mundiale.macro;'
                      'PASSWORD=MM@6e789d1f;'
                      'OPTION=3;')
cursor = conn.cursor()
try:
    data = pd.read_excel( r"C:\Users\igor.malta\Downloads\resultado.xlsx" , sheet_name = 'disparo_wpp')
    data = data.data_engajado.max()
except:
    data = input("coloque uma data no formato yyyy-mm-dd: ")
sql = f"""SELECT distinct
	sale.id_operation,
    eng.operacao,
    eng.id_ticket,
    max(eng.dt_criacao) as data_engajado,
    eng.nome_cliente,
    eng.cpf,
    concat(concat("https://api.whatsapp.com/send/?phone=55", (CASE WHEN canal_atendimento = "WHATSAPP" THEN eng.telefone_id_lead ELSE ifnull(IFNULL(telefone1,telefone2),null) end)),
    CONCAT(CONCAT(CONCAT(CONCAT("&text=Olá! ", eng.nome_cliente) , " souda "), eng.operacao), ". Precisamos que me envie as fotos, para que possamos finalizar o seu pedido. É necessário que envie três fotos no APP BRISA CLIENTES, uma selfie com o documento pessoal em mãos, uma foto da frente e outra de trás do documento RG e CPF para completar seu pedido.
Link passo a passo: https://bit.ly/brisanet_app")) as link
	
FROM wit_core_sale.resume_sale_data as sale
left join core_attendance.tickets_engajados_detalhes as eng on eng.id_ticket = sale.id_ticket
WHERE sale.id_operation = 77
AND sale.tag = "PENDENCIA APP"
AND nome_cliente <> ''
AND eng.dt_criacao > '{data}'
group by 1,2,3,5,6,7
"""

planilha = pd.read_sql(sql, conn)
#Abertura do navegador e busca pelo site
driver = webdriver.Chrome(r"chromedriver.exe")
driver.get("https://web.whatsapp.com/")

sleep (10) #tempo pra logar no wpp com o qrcode

#Percorre todos os links
for x in planilha.link:
    #Busca o link daquela linha
    driver.get(x)
    #clica no snippet para iniciar conversa
    driver.find_element(By.XPATH, '//*[@id="action-button"]').click()
    driver.execute_script('document.querySelector("#fallback_block > div > div > h4:nth-child(2) > a").click()')
    #Aguarda o carregamento da pagina verificando a visibilidade do elemento da tela do wpp
    WebDriverWait(driver, 40).until(expected_conditions.presence_of_element_located((By.CSS_SELECTOR, '#main > footer > div._2BU3P.tm2tP.copyable-area > div > span:nth-child(2) > div > div._2lMWa > div._3HQNh._1Ae7k > button > span')))
    sleep(1)    #tempo para um comando nao atropelar o outro
    #driver.find_element(By.CSS_SELECTOR,'#main > footer > div._2BU3P.tm2tP.copyable-area > div > span:nth-child(2) > div > div._2lMWa > div.p3_M1').send_keys(y, Keys.ENTER)
    #Envia a mensagem
    driver.execute_script('document.querySelector("#main > footer > div._2BU3P.tm2tP.copyable-area > div > span:nth-child(2) > div > div._2lMWa > div._3HQNh._1Ae7k > button > span").click()')
    #inclui o dado OK no dataframe que colocaremos no excel
    sleep(2)    #tempo para um comando nao atropelar o outro
    planilha[["enviado?"]]= "ok"
#Após toda a planilha percorrida, criamos um novo arquivo com os resultados.
planilha.to_excel( r"C:\Users\igor.malta\Downloads\resultado.xlsx" , sheet_name = 'disparo_wpp') 
conn.close()
        

C:\Users\IGOR~1.MAL\AppData\Local\Temp/ipykernel_17912/3438959603.py:43: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(r"chromedriver.exe")


digite a data no formato: yyyy-mm-dd2022-06-01


Timestamp('2022-07-06 18:35:52')